In [1]:
from datasets import load_dataset
train_dataset = load_dataset("imdb", split="train").to_pandas()
test_dataset = load_dataset("imdb", split="test").to_pandas()

Reusing dataset imdb (C:\Users\PION\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (C:\Users\PION\.cache\huggingface\datasets\imdb\plain_text\1.0.0\e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [2]:
test_dataset

,text,label
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
...,...,...
24995,I occasionally let my kids watch this garbage ...,0
24996,When all we have anymore is pretty much realit...,0
24997,The basic genre is a thriller intercut with an...,0
24998,Four things intrigued me as to this film - fir...,0


In [3]:
test_label = test_dataset.groupby('label')
test_label.count()

,text
label,
0,12500
1,12500


In [4]:
train_label = train_dataset.groupby('label')
train_label.count()

,text
label,
0,12500
1,12500


In [5]:
test_dataset[test_dataset['label'] == 0].count()

text     12500
label    12500
dtype: int64

In [6]:
train_dataset[train_dataset['label'] == 0].count()

text     12500
label    12500
dtype: int64

In [7]:
train_dataset['text'] = train_dataset['text'].str.lower()
test_dataset['text'] = test_dataset['text'].str.lower()
test_dataset

,text,label
0,i went and saw this movie last night after bei...,1
1,actor turned director bill paxton follows up h...,1
2,as a recreational golfer with some knowledge o...,1
3,"i saw this film in a sneak preview, and it is ...",1
4,bill paxton has taken the true story of the 19...,1
...,...,...
24995,i occasionally let my kids watch this garbage ...,0
24996,when all we have anymore is pretty much realit...,0
24997,the basic genre is a thriller intercut with an...,0
24998,four things intrigued me as to this film - fir...,0


In [8]:
from string import punctuation
train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')
test_dataset
#Cleaning dataset (part 1)

C:\Users\PION\AppData\Local\Temp\ipykernel_6684\311460489.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
C:\Users\PION\AppData\Local\Temp\ipykernel_6684\311460489.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')


,text,label
0,i went and saw this movie last night after bei...,1
1,actor turned director bill paxton follows up h...,1
2,as a recreational golfer with some knowledge o...,1
3,i saw this film in a sneak preview and it is d...,1
4,bill paxton has taken the true story of the 19...,1
...,...,...
24995,i occasionally let my kids watch this garbage ...,0
24996,when all we have anymore is pretty much realit...,0
24997,the basic genre is a thriller intercut with an...,0
24998,four things intrigued me as to this film firs...,0


In [9]:
#Creating our model
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#TfidfVectorizer is capable of doing both word count vectorization and IDF calculations.
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics
from sklearn.metrics import accuracy_score
import numpy as np

In [26]:
text_pip = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB()) ])
text_pip.fit(train_dataset['text'],train_dataset['label'])
pred = text_pip.predict(test_dataset['text'])

In [11]:
metrics.confusion_matrix(test_dataset['label'], pred)

array([[10989,  1511],
       [ 3057,  9443]], dtype=int64)

In [12]:
print('Accuracy is ' + str(np.mean(pred == test_dataset.label)))
#Pas compris si je dois tester aussi sur les données d'entrainement

Accuracy is 0.81728


## Mots les plus important

In [40]:
wordscount = text_pip[0].get_feature_names()
importance = text_pip[1].feature_log_prob_
importance

array([[-13.77050587, -14.17597098, -14.17597098, ..., -14.17597098,
        -14.86911816, -14.17597098],
       [-14.89530455, -14.89530455, -14.89530455, ..., -14.89530455,
        -14.20215737, -14.89530455]])

In [54]:
from heapq import nlargest
k = 5
nmax = nlargest(k, enumerate(importance[0]), key=lambda x:x[1])
nmax

[(106708, -2.8710788014787916),
 (6098, -3.664771591693647),
 (75439, -3.7310170606305437),
 (108395, -3.7323126144425807),
 (55589, -4.0527055315651275)]

In [56]:
j = 0
while j < k:
    print(wordscount[nmax[j][0]])
    j = j+1

the
and
of
to
is


## Why is accuracy a sufficient measure of evaluation here?
#### L'accuracy est une bonne mesure dans notre cas puisque notre dataset est bien balance. Chaque type de label possede un nobre similaire de donnée permettant une meilleur representation


In [18]:
#nltk.download()

In [19]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [20]:
stops = set(stopwords.words('english'))


In [21]:
train_dataset['text_no_stop'] = train_dataset['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stops)]))
test_dataset['text_no_stop'] = test_dataset['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stops)]))
train_dataset

,text,label,text_no_stop
0,bromwell high is a cartoon comedy it ran at th...,1,bromwell high cartoon comedy ran time programs...
1,homelessness or houselessness as george carlin...,1,homelessness houselessness george carlin state...
2,brilliant overacting by lesley ann warren best...,1,brilliant overacting lesley ann warren best dr...
3,this is easily the most underrated film inn th...,1,easily underrated film inn brooks cannon sure ...
4,this is not the typical mel brooks film it was...,1,typical mel brooks film much less slapstick mo...
...,...,...,...
24995,towards the end of the movie i felt it was too...,0,towards end movie felt technical felt like cla...
24996,this is the kind of movie that my enemies cont...,0,kind movie enemies content watch time bloody t...
24997,i saw descent last night at the stockholm film...,0,saw descent last night stockholm film festival...
24998,some films that you pick up for a pound turn o...,0,films pick pound turn rather good 23rd century...


In [57]:
text_pip.fit(train_dataset['text_no_stop'],train_dataset['label'])
pred = text_pip.predict(test_dataset['text_no_stop'])
metrics.confusion_matrix(test_dataset['label'], pred)
print('Accuracy is ' + str(np.mean(pred == test_dataset.label)))

Accuracy is 0.82712
